In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
path = r'D:\alessia\Instacart basket Analysis - Alessia'

In [4]:
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customer_ords_prods_merged.pkl'))

In [5]:
df.shape

(32404859, 34)

In [6]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


# 2. Consider any security implications that might exist for this new data. You’ll need to address any PII data in the data before continuing your analysis.

***The columns first name and surname need to be removed***

In [7]:
df.drop(['First Name', 'Surname'], axis = 1, inplace = True)

In [8]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_freq,frequency_flag,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


# 03. Customer Behaviour - Regional Segmentation

**categorize regions**

In [9]:
northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
south = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
west = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

**assign regions depending on state**

In [10]:
df.loc[df['State'].isin(northeast), 'Region'] = 'Northeast'
df.loc[df['State'].isin(midwest), 'Region'] = 'Midwest'
df.loc[df ['State'].isin(south), 'Region'] = 'South'
df.loc[df ['State'].isin(west), 'Region'] = 'West'

**quick control check**

In [11]:
df['Region'].value_counts(dropna = False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: Region, dtype: int64

**crosstab of spending habits**

In [12]:
region_spend = pd.crosstab(df['Region'], df ['spending_flag'], dropna = False)

In [13]:
region_spend.to_clipboard()

In [14]:
region_spend

spending_flag,High spender,Low spender
Region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


***Comment: numbers above do not give a clear picture of the spending habits. Adding a percentage calculation***

In [15]:
region_spend['Total'] = region_spend.sum(axis=1)
region_spend['% High spender'] = 100 * region_spend['High spender'] / region_spend ['Total']
region_spend ['% Low spender'] = 100 * region_spend ['Low spender'] / region_spend ['Total']

In [16]:
region_spend

spending_flag,High spender,Low spender,Total,% High spender,% Low spender
Region,,,,,
Midwest,155975,7441350,7597325,2.053025,97.946975
Northeast,108225,5614511,5722736,1.891141,98.108859
South,209691,10582194,10791885,1.943043,98.056957
West,160354,8132559,8292913,1.933627,98.066373


***Comment: spending habits seem quite similar across regions. The Midwest has a very slight tendency to have customers with a higher spending habit.***

# 4. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.

In [17]:
df.loc[df['max_order']>=5,'activity_of_customer']='High order customer'
df.loc[df['max_order']<5,'activity_of_customer']='Low order customer'

In [18]:
df['activity_of_customer'].value_counts(dropna=False)

High order customer    30964564
Low order customer      1440295
Name: activity_of_customer, dtype: int64

In [19]:
df_low=df[df['activity_of_customer']=='Low order customer']

In [20]:
df_low.to_pickle(os.path.join(path,'02 Data','Prepared Data','df_low_order.pkl'))

In [21]:
df_high=df[df['activity_of_customer']=='High order customer']

In [22]:
df_high.to_pickle(os.path.join(path,'02 Data','Prepared Data','df_high_order.pkl'))

In [23]:
df_high


,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,Gender,State,Age,date_joined,n_dependants,fam_status,income,Region,activity_of_customer
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High order customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High order customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High order customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High order customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,South,High order customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404850,3308056,106143,22,4,20,10.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High order customer
32404851,2988973,106143,23,2,22,5.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High order customer
32404852,930,106143,24,6,12,4.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High order customer
32404853,467253,106143,25,6,16,7.0,19675,1,1,Organic Raspberry Black Tea,...,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755,West,High order customer


**merging departments_wrangled with df_high**

In [24]:
df_dep = pd.read_csv((os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv')), index_col = False)

In [25]:
df_dep

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [26]:
df_dep.rename(columns={'Unnamed: 0':'department_id'},inplace=True)

In [27]:
df_final=df_high.merge(df_dep,on='department_id')

In [28]:
df_final.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'final.pkl'))